# Evaluation

## Import Libraries

In [ ]:
from pathlib import Path

import torch
import psutil
import numpy as np
import pandas as pd
import seaborn as sns
import shap
import matplotlib.pyplot as plt
import torch.nn.functional as F
import os
from petastorm import make_reader
from petastorm.pytorch import DataLoader
from sklearn import metrics

from pathlib import Path

from ml.vae import VAE
from ml.ae import AE

## Configuration

In [ ]:
model_dir = 'model_ae_mse/scan44_model/'
model_name = 'ae_scan44_fnn.model'
model_path = model_dir + model_name
data_path = 'model_input_scan44_fnn/test.model_input.parquet'
results_dir = 'results_test'
# get number of cores
#num_cores = psutil.cpu_count(logical=True)
num_cores = 8
pos_label = 'scan44'
model_type = AE

## Load Model

In [ ]:
model = model_type.load_from_checkpoint(checkpoint_path=model_path, map_location=torch.device('cuda'))

model.eval()

## Define Reconstruct Error Function

In [ ]:
def calc_recon_loss(recon_x, x, logvar = None, mu = None, loss_type: str = 'mse') -> list:
    """
    Return the reconstruction loss

    :param recon_x: reconstructed x, output from model
    :param x: original x
    :param logvar: variance, output from model, ignored when loss_type isn't 'bce+kd'
    :param mu: mean, output from model, ignored when loss_type isn't 'bce+kd'
    :param loss_type: method to compute loss, option: 'bce', 'mse', 'bce+kd'
    :return: list of reconstruct errors
    :rtype: list
    """

    loss_type = loss_type.lower()

    # 73 is the number of features
    NUM_FEATURES=73
    if loss_type == 'mse':
        recon_error = F.mse_loss(recon_x, x, reduction='none').view(-1, NUM_FEATURES).mean(dim=1)
    elif loss_type == 'mse+kd':
        mse = F.mse_loss(recon_x, x, reduction='none').view(-1, NUM_FEATURES).mean(dim=1)
        kd = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        recon_error = mse + kd
    else:
        raise Exception('Invalid loss type: only support "mse", or "mse+kd"')

    return recon_error.tolist()

## Calculate Reconstruction Error

In [ ]:
reader = make_reader(
Path(data_path).absolute().as_uri(), reader_pool_type='process', workers_count=num_cores,
pyarrow_serialize=True, num_epochs=1
)

In [ ]:
def calc_recon_loss_batch(loss_type):
    #reader.reset()
    # No shuffle
    dataloader = DataLoader(reader, batch_size=3000)

    loss_list = []
    label_list = []

    for data in dataloader:
        x = data['feature']
        label = data['label']
        recon_x, mu, logvar = model(x)

        loss = calc_recon_loss(recon_x, x, logvar, mu, loss_type=loss_type)

        loss_list.extend(loss)
        label_list.extend(label)
    
    return loss_list, label_list

In [ ]:
save_dir = model_dir + results_dir + '/' + os.path.splitext(model_name)[0]
Path(save_dir).mkdir(parents=True, exist_ok=True)
loss_name = 'loss.npy'
labels_name = 'labels.npy'
if (model_type == AE):
    print('AE')
    loss, label = calc_recon_loss_batch('mse')
elif (model_type == VAE):
    print('VAE')
    loss, label = calc_recon_loss_batch('mse')
#np.save(save_dir + '/' + 'mse_loss.npy', np.array(mse_loss_list))
#np.save(save_dir + '/' + 'bce_loss.npy', np.array(bce_loss_list))
#np.save(save_dir + '/' + 'bce_kd_loss.npy', np.array(bce_kd_loss_list))
#labels1 = np.array(list(map(lambda x: x == pos_label, label_list)))
np.save(save_dir + '/' + loss_name, np.array(loss))
labels_list = list(map(lambda x: x == pos_label, label))
labels_np = np.array(labels_list)
np.save(save_dir + '/' + labels_name, labels_np)
print('Data size:', labels_np.size)
print('Condition positive:', np.sum(labels_np))
#np.save(save_dir + '/' + 'labels.npy', labels1)

In [ ]:
# load the numpy arrays
scores_run1 = np.load(save_dir + '/loss_run1.npy')
scores_run2 = np.load(save_dir + '/loss_run2.npy')
scores_run3 = np.load(save_dir + '/loss_run3.npy')

labels_run1 = np.load(save_dir + '/labels_run1.npy')
labels_run2 = np.load(save_dir + '/labels_run2.npy')
labels_run3 = np.load(save_dir + '/labels_run3.npy')

In [ ]:
np.where(labels_run1 == True)

In [ ]:
np.where(labels_run2 == True)

In [ ]:
np.where(labels_run3 == True)

In [ ]:
# check if all the labels are equal
equate_list1 = list(map(lambda x, y: x == y, labels_run1, labels_run2))
equate_list2 = list(map(lambda x, y: x == y, labels_run1, labels_run3))
equate_list3 = list(map(lambda x, y: x == y, labels_run2, labels_run3))
print(np.sum(labels_run1))
print(np.sum(labels_run2))
print(np.sum(labels_run3))

In [ ]:
import functools
functools.reduce(lambda x, y: x and y, equate_list3)

In [ ]:
labels1 = np.array(list(map(lambda x: x == pos_label, label_list)))
print('Data size:', labels1.size)
print('Condition positive:', np.sum(labels1))
#np.save(save_dir + '/' + 'labels.npy', labels1)

In [ ]:
labels2 = np.array([x == 'nerisbotnet' for x in label_list])

In [ ]:
labels3 = np.array([x == 'dos' for x in label_list]) 

In [ ]:
equate_list1 = list(map(lambda x, y: x == y, labels1, labels2))
equate_list2 = list(map(lambda x, y: x == y, labels1, labels3))

In [ ]:
import functools
functools.reduce(lambda x, y: x and y, equate_list1)

In [ ]:
functools.reduce(lambda x, y: x and y, equate_list2)

In [ ]:
np.sume

In [ ]:
save_dir = model_dir + 'results_val' + '/' + os.path.splitext(model_name)[0]
with open('mse_loss.txt', 'w') as file:
    file.write('\n'.join(mse_loss_list))

## Construct a Pandas Dataframe for Easier Evaluation

In [ ]:
df = pd.DataFrame(
    {
        'x': x_list,
        'mse_loss': mse_loss_list,
        'bce_loss': bce_loss_list,
        'bce+kd_loss': bce_kd_loss_list,
        'label': label_list
    }
)

## Plot Function

### Plot for ROC

In [ ]:
def plot_roc(df: pd.DataFrame, malicious_type: str):
    malicious_type_set = {'anomaly-spam', 'blacklist', 'dos', 'nerisbotnet', 'scan11', 'scan44'}

    if malicious_type not in malicious_type_set:
        raise Exception(f'Invalid malicious_type, only support "{malicious_type_set}"')

    part_df = df[(df['label'] == 'background') | (df['label'] == malicious_type)]
    label = (
        part_df
            .label.replace({
                'background': 0,
                malicious_type: 1,
            })
        .tolist()
    )


    mse_loss = part_df.mse_loss.tolist()
    bce_loss = part_df.bce_loss.tolist()
    bce_kd_loss = part_df['bce+kd_loss'].tolist()

    fig, ax = plt.subplots(figsize=(5, 5))

    fpr_mse, tpr_mse, thresholds_mse = metrics.roc_curve(label, mse_loss)
    fpr_bce, tpr_bce, thresholds_bce = metrics.roc_curve(label, bce_loss)
    fpr_bce_kd, tpr_bce_kd, thresholds_bce_kd = metrics.roc_curve(label, bce_kd_loss)

    auc_mse = metrics.auc(fpr_mse, tpr_mse)
    auc_bce = metrics.auc(fpr_bce, tpr_bce)
    auc_bce_kd = metrics.auc(fpr_bce_kd, tpr_bce_kd)

    ax.plot([0, 1], [0,1], 'k--')
    ax.plot(fpr_mse, tpr_mse, label=f'with mse loss (auc = {auc_mse: .2f})')
    ax.plot(fpr_bce, tpr_bce, label=f'with bce loss (auc = {auc_bce: .2f})')
    ax.plot(fpr_bce_kd, tpr_bce_kd, label=f'with bce+kd loss (auc = {auc_bce_kd: .2f})')

    ax.set_xlabel('False positive rate')
    ax.set_ylabel('True positive rate')

    ax.set_title(f'ROC of background and {malicious_type}')

    ax.legend(loc='lower right')

    fig.show()

### Plot for KDE

In [ ]:
def plot_kde(df: pd.DataFrame, loss_type: str, malicious_type: str):
    loss_type_set = {'mse', 'bce', 'bce+kd'}
    if loss_type not in loss_type_set:
        raise Exception(f'Invalid loss_type, only support "{loss_type}"')

    malicious_type_set = {'anomaly-spam', 'blacklist', 'dos', 'nerisbotnet', 'scan11', 'scan44'}
    if malicious_type not in malicious_type_set:
        raise Exception(f'Invalid malicious_type, only support "{malicious_type_set}"')

    normal_recon_error = df[df['label'] == 'background'][f'{loss_type}_loss'].tolist()
    malicious_recon_error = df[df['label'] == malicious_type][f'{loss_type}_loss'].tolist()

    fig, ax = plt.subplots(figsize=(5, 5))
    sns.kdeplot(
        normal_recon_error,
        ax=ax,
        label=f'background {loss_type} loss'
    )
    sns.kdeplot(
        malicious_recon_error,
        ax=ax,
        label=f'{malicious_type} {loss_type} loss'
    )

    ax.set_title(f'Reconstruction Error Distribution of background traffic and {malicious_type}')
    ax.legend(loc='lower right')

    fig.show()

### Plot for Gradient

In [ ]:
def plot_gradient(df: pd.DataFrame, malicious_type: str, model: VAE):
    malicious_type_set = {'anomaly-spam', 'blacklist', 'dos', 'nerisbotnet', 'scan11', 'scan44'}

    if malicious_type not in malicious_type_set:
        raise Exception(f'Invalid malicious_type, only support "{malicious_type_set}"')

    x = torch.FloatTensor(df[df['label'] == malicious_type]['x'].tolist())

    # clear gradient
    model.zero_grad()

    # get model output
    recon_x, logvar, mu = model(x)

    # calculate loss
    loss = model.loss_function(recon_x, x, mu, logvar)

    # get the gradient w.r.t loss
    grad = torch.autograd.grad(loss, recon_x, retain_graph=False)[0].view(-1, NUM_FEATURES)

    # get selected feature grad only
    grad = grad[:, [11, 13, 46, 53, 0, 1, 6, 3, 2, 4, 9, 14, 30, 57, 42, 43]]

    # build selected feature name
    feature_name = [
        'entropy_dst_ip', 'entropy_dst_port', 'dst_SMTP', 'dst_HTTP', 'mean_duration', 'mean_packet', 'std_packet',
        'mean_packet_rate', 'mean_num_of_bytes', 'mean_byte_rate', 'std_byte_rate', 'entropy_flags', 'src_RPC',
        'dst_RPC', 'dst_FTP_20', 'dst_FTP_21'
    ]

    fig, ax = plt.subplots(figsize=(10, 10))

    sns.barplot(x=grad.T.reshape(-1).tolist(), y=feature_name * grad.shape[0], orient='h', ax=ax)

    ax.set_title(f'Gradient of {malicious_type}')

    fig.show()

## Plot ROC with Different Loss

### anomaly-spam

In [ ]:
plot_roc(df, 'anomaly-spam')

### blacklist

In [ ]:
plot_roc(df, 'blacklist')

### dos

In [ ]:
plot_roc(df, 'dos')

### nerisbotnet

In [ ]:
plot_roc(df, 'nerisbotnet')

### scan44

In [ ]:
plot_roc(df, 'scan44')

### scan11

In [ ]:
plot_roc(df, 'scan11')

## Plot Reconstruction Error Distribution

### anomaly-spam

In [ ]:
plot_kde(df, 'mse', 'anomaly-spam')

### blacklist

In [ ]:
plot_kde(df, 'mse', 'blacklist')

### dos

In [ ]:
plot_kde(df, 'mse', 'dos')

### nerisbotnet

In [ ]:
plot_kde(df, 'mse', 'nerisbotnet')

### scan44

In [ ]:
plot_kde(df, 'mse', 'scan44')

### scan11

In [ ]:
plot_kde(df, 'mse', 'scan11')

## Gradient Explainer

### anomaly-spam

In [ ]:
plot_gradient(df, 'anomaly-spam', model)

### blacklist

In [ ]:
plot_gradient(df, 'blacklist', model)

### dos

In [ ]:
plot_gradient(df, 'dos', model)

### nerisbotnet

In [ ]:
plot_gradient(df, 'nerisbotnet', model)

### scan44

In [ ]:
plot_gradient(df, 'scan44', model)

### scan11

In [ ]:
plot_gradient(df, 'scan11', model)